In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# hyperparameter
batch_size = 4
block_size = 8
max_iters = 10000
eval_iters = 1000
eval_interval = 300

learning_rate = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(1337)


In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-26 16:27:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-07-26 16:27:50 (26.4 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [4]:
# read dataset
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
# Mapping sets
stoi = {ch:i for i ,ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
# encode str -> int
encode = lambda s:[stoi[c] for c in s]
# decode int -> str
decode = lambda l:''.join([itos[i] for i in l])

In [5]:
data = torch.tensor(encode(text), dtype = torch.long)
# split into train and val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,)) # predicting the 1st encoded index
  # print(f"the ix is :{ix}")
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


In [6]:
class BiagramModel(nn.Module):

  def __init__(self, vocab_size) -> None:
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, target= None):

    logits = self.token_embedding_table(idx)
    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)

      loss = F.cross_entropy(logits, target) # special case for multiple dimensions

    return logits , loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      # getting the predictions
      logits, loss = self(idx)
      # focus on last time step
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim = -1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim = 1)# (B, T+1)
    return idx

In [7]:
model = BiagramModel(vocab_size)
m = model.to(device)
# Model optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [10]:
# Training loop
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step{iter}: train loss {losses['train']:.4f}, val_loss {losses['val']:.4f}")

  xb, yb = get_batch('train')

  # Evaluate loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none= True)
  loss.backward()
  optimizer.step()

step0: train loss 2.5467, val_loss 2.5557
step300: train loss 2.5353, val_loss 2.5525
step600: train loss 2.5228, val_loss 2.5419
step900: train loss 2.5138, val_loss 2.5284
step1200: train loss 2.5414, val_loss 2.5399
step1500: train loss 2.5113, val_loss 2.5305
step1800: train loss 2.5090, val_loss 2.5199
step2100: train loss 2.5245, val_loss 2.5231
step2400: train loss 2.5065, val_loss 2.5265
step2700: train loss 2.5064, val_loss 2.5062
step3000: train loss 2.5066, val_loss 2.5301
step3300: train loss 2.4903, val_loss 2.5136
step3600: train loss 2.4979, val_loss 2.5069
step3900: train loss 2.4897, val_loss 2.4925
step4200: train loss 2.4925, val_loss 2.4980
step4500: train loss 2.4951, val_loss 2.4852
step4800: train loss 2.4903, val_loss 2.4964
step5100: train loss 2.4821, val_loss 2.5022
step5400: train loss 2.4792, val_loss 2.4865
step5700: train loss 2.4783, val_loss 2.4802
step6000: train loss 2.4976, val_loss 2.4931
step6300: train loss 2.4762, val_loss 2.4882
step6600: train 

In [11]:
# generate from model
context = torch.zeros((1,1), dtype= torch.long, device = device)
print(decode(m.generate(context, max_new_tokens= 100)[0].tolist()))


Anou trif werue.
K berd owhow noe, han,
TInolidyowathltay$SCERLOVk ploully im.
GA Gaume f ler'e: me 
